sources:
docs.streamlit.io/library/get-started

# Introduction

In [29]:
# Import Libraries
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_any_dtype,
    is_numeric_dtype,
    is_object_dtype,
)

import plotly.figure_factory as ff

## Streamlit

In [2]:
import streamlit as st

In [3]:
st.set_page_config(
    page_title="9base",
    page_icon="⚾",
    layout="wide",
    initial_sidebar_state="expanded",
    menu_items={
        'Get Help': 'https://www.extremelycoolapp.com/help',
        'Report a bug': "https://www.extremelycoolapp.com/bug",
        'About': "# This is a header. This is an *extremely* cool app!"
    }
)

In [19]:
#from st_pages import Page, Section, show_pages, add_page_title

##⚾🥇

#add_page_title() # By default this also adds indentation

## Specify what pages should be shown in the sidebar, and what their titles and icons should be
#show_pages(
#    [
#        Section("14 Day", icon="🥇"),
#        Section("Season", icon="♂️"),
#        Page("player_batting_season.py", "Player Batting", "🏠"),
#    ]
#)

## Load Data

In [4]:
# Format for GitHub
df = pd.read_csv('data_storage/player_batting_season.csv')

# Format for Jupyter Notebook
#df = pd.read_csv('C:/Users/b7tbu/NINEBASE/ninebase/data_storage/player_batting_season.csv')

In [6]:
df.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,...,KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,18401,2023,Ronald Acuna Jr.,ATL,25,86,347,396,117,71,24,1,21,78,54,43,2,49,4,2,0,7,41,7,0.33700,150,91,59,6,1528,617,911,16,0,0,0.10900,0.12400,0.88000,0.41400,0.59400,1.00800,0.25600,0.34400,0.01650,0.19700,0.50000,0.30300,0.06600,0.23100,0.10700,0.00000,0.42700,35.50000,83,33.50000,0.30000,12.20000,-3.90000,48.70000,4.90000,$39.0,6.80000,168,3.67000,-5.24000,8.91000,42.05000,4.33000,0.90000,NaN,0,3.65000,0.43000,0.50100,93.70000,0.23200,85.10000,0.06200,89.30000,0.08200,80.00000,0.09600,85.40000,0.02700,85.00000,NaN,NaN,0.00800,NaN,17.40000,11.30000,1.00000,6.40000,-0.90000,2.30000,NaN,2.28000,3.20000,1.09000,5.09000,...,NaN,0.00100,0.19800,0.23700,NaN,85.40000,NaN,79.60000,94.10000,89.60000,84.80000,NaN,81.90000,94.40000,85.30000,NaN,-1.50000,NaN,2.90000,-1.90000,0.70000,-4.70000,NaN,-6.60000,-6.80000,2.30000,NaN,2.50000,NaN,-7.30000,8.20000,3.90000,0.50000,NaN,-4.10000,4.20000,0.10000,NaN,-0.30000,NaN,4.10000,12.80000,0.90000,2.00000,NaN,0.10000,3.90000,10.50000,NaN,-0.20000,NaN,3.34000,2.87000,0.96000,4.97000,NaN,4.27000,1.31000,2.93000,NaN,0.22600,0.67500,0.45600,0.65100,0.87700,0.82200,0.51200,18.10000,NaN,134,122,55,128,143,157,114,0.97000,116,82,184,112,97,85,64,89,135,94.90000,7.80000,47,0.15700,116.70000,165,0.55000,300,0.14000,0.22100,0.35400,0.65900,0.46100,4.80000
1,19755,2023,Shohei Ohtani,LAA,28,87,334,389,99,48,15,5,31,61,68,47,4,86,1,2,0,7,11,4,0.29600,110,102,38,6,1541,604,937,7,0,0,0.12100,0.22100,0.55000,0.38300,0.65000,1.03300,0.35300,0.31100,0.01080,0.15200,0.44000,0.40800,0.05900,0.30400,0.06400,0.00000,0.42700,34.90000,82,35.20000,NaN,12.00000,-9.40000,39.50000,4.00000,$31.7,6.20000,177,2.92000,-6.26000,9.18000,36.48000,3.81000,1.03000,NaN,0,3.71000,-0.88000,0.40600,94.50000,0.21700,84.60000,0.10800,89.70000,0.08600,80.30000,0.14100,86.20000,0.04200,87.50000,NaN,NaN,0.01400,NaN,12.90000,6.90000,3.50000,4.50000,5.00000,1.20000,NaN,2.07000,2.08000,2.12000,3.43000,...,NaN,NaN,0.10200,0.24000,NaN,86.30000,NaN,80.10000,94.60000,90.70000,87.00000,NaN,NaN,94.10000,84.70000,NaN,-7.30000,NaN,3.50000,-1.70000,1.20000,-6.70000,NaN,NaN,-1.80000,0.60000,NaN,2.80000,NaN,-7.20000,8.50000,3.90000,0.70000,NaN,NaN,4.10000,-0.40000,NaN,4.10000,NaN,4.50000,6.50000,1.90000,1.50000,NaN,NaN,5.40000,7.90000,NaN,1.92000,NaN,3.53000,1.44000,1.21000,2.36000,NaN,NaN,3.45000,2.15000,NaN,0.29700,0.67900,0.47900,0.51900,0.80800,0.71400,0.47500,18.70000,NaN,121,144,97,121,160,220,105,0.76000,104,108,252,97,103,102,65,95,124,93.70000,13.10000,46,0.18000,117.10000,130,0.51000,255,0.13000,0.26500,0.29100,0.64100,0.42600,4.00000
2,13611,2023,Mookie Betts,LAD,30,84,325,386,88,42,22,1,23,68,57,52,0,64,4,5,0,4,7,2,0.27100,79,130,57,12,1482,620,862

## Scatter Chart

In [7]:
import plotly.express as px

In [8]:
fig = px.scatter(
    df.query("Season==2023"),
    x = "wRC+",
    y = "Hard%+",
    color = "Team",
    hover_name = "Name",
    log_x = True,
    size_max = 60,
)

In [9]:
#st.plotly_chart(fig, theme="streamlit", use_container_width = True)

## Filters

In [10]:
# calculate min/max/mean for slider
min_AB = df['AB'].min()
max_AB = df['AB'].max()
mean_AB = df['AB'].mean()

# float em
min_AB = float(min_AB)
max_AB = float(max_AB)
mean_AB = float(mean_AB)

In [11]:
# slider

ab_slider = st.slider('At Bat Range', min_AB, max_AB, (mean_AB, max_AB))


2023-07-08 09:27:04.206 
  command:

    streamlit run C:\Users\b7tbu\anaconda3\envs\ninebase\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [12]:
## Sidebar

## Columns

In [13]:
col1, col2 = st.columns([2,1])

In [14]:
col1.subheader("Scatter Chart")
col1.plotly_chart(fig, theme="streamlit", use_container_width = True)

col2.subheader("Filters")
col2.write(ab_slider)

## Raw Data Section

In [15]:
from st_aggrid import AgGrid

In [31]:
df.columns.tolist()

['IDfg',
 'Season',
 'Name',
 'Team',
 'Age',
 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'GDP',
 'SB',
 'CS',
 'AVG',
 'GB',
 'FB',
 'LD',
 'IFFB',
 'Pitches',
 'Balls',
 'Strikes',
 'IFH',
 'BU',
 'BUH',
 'BB%',
 'K%',
 'BB/K',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'BABIP',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'wOBA',
 'wRAA',
 'wRC',
 'Bat',
 'Fld',
 'Rep',
 'Pos',
 'RAR',
 'WAR',
 'Dol',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'phLI',
 'PH',
 'WPA/LI',
 'Clutch',
 'FB% (Pitch)',
 'FBv',
 'SL%',
 'SLv',
 'CT%',
 'CTv',
 'CB%',
 'CBv',
 'CH%',
 'CHv',
 'SF%',
 'SFv',
 'KN%',
 'KNv',
 'XX%',
 'PO%',
 'wFB',
 'wSL',
 'wCT',
 'wCB',
 'wCH',
 'wSF',
 'wKN',
 'wFB/C',
 'wSL/C',
 'wCT/C',
 'wCB/C',
 'wCH/C',
 'wSF/C',
 'wKN/C',
 'O-Swing%',
 'Z-Swing%',
 'Swing%',
 'O-Contact%',
 'Z-Contact%',
 'Contact%',
 'Zone%',
 'F-Strike%',
 'SwStr%',
 'BsR',
 'FA% (sc)',
 

In [34]:
df_short = df[['Name', 'Team', 'Age', 'AB', 'BB%+', 'K%+', 'BABIP+', 'Hard%+', 'wRC+', 'WAR',]]

In [35]:
st.divider()
st.subheader('Data')
AgGrid(df_short)

AgGridReturn(data=                      Name Team  Age   AB  BB%+  K%+  BABIP+  Hard%+  wRC+  \
0         Ronald Acuna Jr.  ATL   25  347   122   55     114     135   168   
1            Shohei Ohtani  LAA   28  334   144   97     105     124   177   
2             Mookie Betts  LAD   30  325   152   74      89     134   150   
3            Wander Franco  TBR   22  331    95   59     103     109   125   
4           Corbin Carroll  ARI   22  297   101   88     106     100   145   
5              Luis Robert  CHW   25  327    69  123     109     106   144   
6          Freddie Freeman  LAD   33  347   113   76     119     121   152   
7                Juan Soto  SDP   24  298   238   89     105     117   153   
8       Fernando Tatis Jr.  SDP   24  282    90   85     104     124   139   
9             Jose Ramirez  CLE   30  325   128   42      97      93   132   
10           Adolis Garcia  TEX   30  336   110  111     100     140   131   
11              Mike Trout  LAA   31  304   15